In [77]:
# Librerias necesarias
import os
import sys
import json
import random
import numpy as np
from collections import deque
# Add the parent directory to sys.path to allow relative imports
sys.path.append(os.path.abspath(os.path.join('..', '1. codigo analisis')))
import parametros as p

In [ ]:
# Cargo los datos necesarios y transformo las llaves nuevamente a int (que se habian vuelto str)
incertidumbre = {}
for hospital in range(0,4): # 0 son llegadas a WL
    incertidumbre[hospital] = {}
    for requerimiento in range(1,4):
        incertidumbre[hospital][requerimiento] = {}

with open("resultados incertidumbre/incertidumbre_simulada.json", "r") as file:
    incertidumbre_keys_str = json.load(file)

for hospital in range(0,4): # 0 son llegadas a WL
    for requerimiento in range(1,4):
        for grd in range(1,9):
            incertidumbre[hospital][requerimiento][grd] = []
            lista = incertidumbre_keys_str[str(hospital)][str(requerimiento)][str(grd)]
            if lista != []:
                for data in lista:
                    arreglado = {
                    'TI': data["TI"],
                    'camino': {1: data['camino']["1"], 2: data['camino']["2"], 3: data['camino']["3"]},
                    'espera': {1: data['espera']["1"], 2: data['espera']["2"], 3: data['espera']["3"]}
                    }
                    incertidumbre[hospital][requerimiento][grd].append(arreglado)
            else:
                incertidumbre[hospital][requerimiento][grd] = []


In [ ]:
class Paciente:
    CONTADOR_ID = 1  # Variable de clase para contar el número de Pacientes creados

    def __init__(self, hospital_llegada, requerimiento_inicial, grd, datos_incertidumbre):
        # Atributos del paciente al llegar, no cambian
        self.id = Paciente.CONTADOR_ID  # Asigna un ID único al Paciente
        self.hospital_llegada = hospital_llegada
        self.requerimiento_inicial = requerimiento_inicial
        self.grd = grd
        self.ti_inicial = datos_incertidumbre["TI"]
        self.camino = datos_incertidumbre["camino"]
        self.espera = datos_incertidumbre["espera"]

        # Diccionarios parametros unidades y hospitales inverso (para simplificar el acceso)
        self.dict_hospitales = self.invertir_dict(p.dict_hospitales)
        self.dict_unidades = self.invertir_dict(p.dict_unidades)

        # Atributos que cambian a lo largo de la simulación
        self.log_eventos = []
        self.ti_evento_actual = datos_incertidumbre["TI"]
        self.tiempo_actual = datos_incertidumbre["TI"]
        self.hospital_actual = hospital_llegada
        self.unidad_actual = self.unidad_llegada()
        self.unidad_requerida = self.requerimiento_inicial
        self.esperando = True # True si esta esperando y no siendo tratado (revisar en cada ciclo)
        self.costo_social = 0
        self.costos_operativos = 0
        Paciente.CONTADOR_ID += 1  # Incrementa el contador de Pacientes

    # Necesarias para cuando se crean los pacientes
    def invertir_dict(self, diccionario):
        invertido = {v: k for k, v in diccionario.items()}
        return invertido
    
    def unidad_llegada(self):
        if self.hospital_llegada == 0:
            unidad_actual = p.dict_unidades["WL"]
        else:
            unidad_actual = p.dict_unidades["ED"]
        return unidad_actual
    
    # Necesarias durante la simulación
    def agregar_log_evento(self):
        if self.hospital_actual == 0:
            hospital = "WL"
        else:
            hospital = self.dict_hospitales[self.hospital_actual]
        new_row = {
            'ID': self.id,
            'MS_GRD': self.grd,
            'UBICACIÓN': hospital + "_" + self.unidad_actual,
            'TI': self.ti_evento_actual,
            'TF': self.tiempo_actual,
            'HOSPITAL': hospital,
            'UNIDAD': self.dict_unidades[self.unidad_actual]
        }
        self.log_eventos.append(new_row)
    
    def costo_espera(self, tiempo_espera = 1): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_espera_wl[self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["GA"]: 
            costo = p.dict_costo_espera_ga[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual == p.dict_unidades["ED"]: 
            costo = p.dict_costo_espera_ed[self.hospital_actual][self.grd][self.requerimiento_inicial] * tiempo_espera
        elif self.unidad_actual in (p.dict_unidades["OR"], p.dict_unidades["ICU"], p.dict_unidades["SDU/WARD"]): # (1, 2, 3)
            costo = p.dict_costo_espera_hospitalizado[self.hospital_actual][self.grd][self.unidad_actual][self.unidad_requerida] * tiempo_espera
        else:
            costo = 0
        return costo

    def costo_traslado(self, hospital_destino): # Revisado, funciona bien
        costo = p.dict_costo_traslado[self.hospital_actual][hospital_destino][self.grd][self.requerimiento_inicial]
        return costo
    
    def costo_desvio(self): # Revisado, funciona bien
        if self.unidad_actual == p.dict_unidades["ED"]:
            costo = p.dict_costo_derivar_ed[self.hospital_actual][self.grd][self.requerimiento_inicial]
        elif self.unidad_actual == p.dict_unidades["WL"]: 
            costo = p.dict_costo_derivar_wl[self.grd][self.requerimiento_inicial]
        return costo

    def __str__(self):
        return f"Paciente {self.id} | GRD: {self.grd} | Hospital actual: {self.hospital_actual} | Unidad: {self.unidad_actual} | Tiempo actual: {self.tiempo_actual}"


In [118]:
# Crear un diccionario con un paciente de cada GRD y requerimiento (para WL)
d_paci = {}
hospital = 1
for grd in range(1, 9):  # GRD values
    d_paci[grd] = {}
    for requerimiento in range(1, 4):  # Requerimiento values
        d_paci[grd][requerimiento] = []
        for cantidad in range(random.randint(1, 5)):  # Cantidad de pacientes por combinación
            # Crear un paciente con datos ficticios para cada combinación
            datos_incertidumbre = {
                "TI": 0,
                "camino": {1: [], 2: [], 3: []},
                "espera": {1: 0, 2: 0, 3: 0}
            }
            paciente = Paciente(hospital_llegada=hospital, requerimiento_inicial=requerimiento, grd=grd, datos_incertidumbre=datos_incertidumbre)
            # paciente.unidad_actual = p.dict_unidades["OR"]
            d_paci[grd][requerimiento].append(paciente)



In [4]:
# Se instancian a todos los pacientes a partir de los datos de incertidumbre
pacientes = {}
lista_pacientes = []
for hospital in range(0,4): # 0 son llegadas a WL
    pacientes[hospital] = {}
    for requerimiento in range(1,4):
        pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for i in range(cantidad_pacientes):
                    paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                    pacientes[hospital][requerimiento][grd].append(paciente)
                    lista_pacientes.append(paciente)
            else:
                pacientes[hospital][requerimiento][grd] = []

# Se generan tantas listas como pacientes haya
def agrupar_pacientes_por_llegada(pacientes):
    pacientes_separados_por_llegada = {}
    for paciente in pacientes:
        dia = paciente.ti_inicial
        if dia not in pacientes_separados_por_llegada:
            pacientes_separados_por_llegada[dia] = []
        pacientes_separados_por_llegada[dia].append(paciente)
    return pacientes_separados_por_llegada

pacientes_separados_por_llegada = agrupar_pacientes_por_llegada(lista_pacientes)

In [ ]:
# Aqui podemos ver los objetos de clase Pacientes que llegaron en x ciclo
x = 2
pacientes_separados_por_llegada.get(x)

In [ ]:
class Simulacion:

    def __init__(self):
        pass

    def __str__(self):
        pass

In [ ]:
class Hospital:

    def __init__(self):
        pass

    def __str__(self):
        pass

In [ ]:
class WL:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    al usar deques las operaciones de append y popleft son O(1), lo cual lo hace mas eficiente
    al ser un sistema de colas FIFO"""
    def __init__(self, requerimientos: list, grds: list):
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Revisado, funciona bien
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
    
    # Siempre el primero de cada sublista es el que lleva mas esperando porque llego antes
    def sacar_paciente(self, requerimiento: int, grd: int): # Revisado, funciona bien
        if self.sub_listas[requerimiento][grd]:
            return self.sub_listas[requerimiento][grd].popleft()
        else:
            return None
    
    def acumular_costo_ciclo(self): # Revisado, funciona bien
        for requerimiento in self.sub_listas:
            for grd in self.sub_listas[requerimiento]:
                if self.sub_listas[requerimiento][grd]:
                    for paciente in self.sub_listas[requerimiento][grd]:
                        self.costo_acumulado_sub_listas[requerimiento][grd] += paciente.costo_espera()
                else:
                    self.costo_acumulado_sub_listas[requerimiento][grd] += 0

    def __str__(self):
        return self.sub_listas, self.costo_acumulado_sub_listas

In [ ]:
class PS:
    """Funciona con varias sublistas, una por cada combinacion de requerimiento y grd,
    funciona simplemente como un sumidero de pacientes"""
    def __init__(self, requerimientos: list, grds: list):
        self.sub_listas = {}
        self.costo_acumulado_sub_listas = {}
        self.crear_sublistas(requerimientos, grds)
    
    def crear_sublistas(self, requerimientos: list, grds: list): # Igual que WL
        for requerimiento in requerimientos:
            self.sub_listas[requerimiento] = {}
            self.costo_acumulado_sub_listas[requerimiento] = {}
            for grd in grds:
                self.sub_listas[requerimiento][grd] = deque()
                self.costo_acumulado_sub_listas[requerimiento][grd] = 0
    
    def agregar_paciente(self, paciente: Paciente): # Revisado, funciona bien
        self.sub_listas[paciente.requerimiento_inicial][paciente.grd].append(paciente)
        self.costo_acumulado_sub_listas[paciente.requerimiento_inicial][paciente.grd] += paciente.costo_desvio() # Revisado, funciona bien

    def __str__(self):
        print("Sublistas de PS:")
        display(self.sub_listas)
        print("Costos acumulados de PS:")
        display(self.costo_acumulado_sub_listas)
        pass

In [ ]:
class Unidad:

    def __init__(self):
        pass

    def __str__(self):
        pass

In [11]:
class Or(Unidad):

    def __init__(self):
        pass

    def __str__(self):
        pass

In [12]:
class Icu(Unidad):

    def __init__(self):
        pass

    def __str__(self):
        pass

In [13]:
class SduWard(Unidad):

    def __init__(self):
        pass

    def __str__(self):
        pass

In [14]:
class GA(Unidad):

    def __init__(self):
        pass

    def __str__(self):
        pass

In [15]:
class ED(Unidad):

    def __init__(self):
        pass

    def __str__(self):
        pass